In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

torch.set_printoptions(sci_mode=False)

In [2]:
lr = 0.0001
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 1000
seed = 165

In [3]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [4]:
G = nx.karate_club_graph()
adj = nx.to_numpy_matrix(G)
adj = np.expand_dims(adj, axis=0)

adj_norm = normalize(adj)

adj = torch.FloatTensor(np.array(adj))
adj_norm = torch.FloatTensor(np.array(adj_norm))

# features
partition = community_louvain.best_partition(G)

#get binary matrix of the partition
nb_community = max(list(partition.values())) + 1
communities =  np.array(list(partition.values())).reshape(-1)
C = np.eye(nb_community)[communities]
features = torch.FloatTensor(C)
features = features.unsqueeze(0)
C = torch.FloatTensor(C).unsqueeze(0)

Q = modularity_matrix(adj)

In [5]:
# Model and optimizer

model = GNN(batch_size=adj.shape[0],
            nfeat=adj.shape[1],
            ndim=nb_community)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    adj_norm = adj_norm.cuda()
    Q = Q.cuda()
    C = C.cuda()

# Train model
t_total = time.time()

optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

In [6]:
for epoch in range(epochs):

    t = time.time()
    model.train()
    optimizer.zero_grad()

    C = model(features, adj_norm)
    loss = modularity(C,Q)

    loss.backward()

    optimizer.step()

    if epoch == 0:
        best_loss = loss
    else:
        if loss < best_loss:
            best_loss = loss

    if epoch % 100 == 0:
        print('Epoch: {:04d}'.format(epoch + 1),
              'Modularity: {:.8f}'.format(-best_loss.item()),
              'time: {:.4f}s'.format(time.time() - t))



print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 Modularity: 0.41559827 time: 0.4361s
Epoch: 0101 Modularity: 0.41583103 time: 0.0000s
Epoch: 0201 Modularity: 0.41620481 time: 0.0087s
Epoch: 0301 Modularity: 0.41724196 time: 0.0008s
Epoch: 0401 Modularity: 0.41724196 time: 0.0008s
Epoch: 0501 Modularity: 0.41724196 time: 0.0000s
Epoch: 0601 Modularity: 0.41724196 time: 0.0008s
Epoch: 0701 Modularity: 0.41724196 time: 0.0000s
Epoch: 0801 Modularity: 0.41724196 time: 0.0075s
Epoch: 0901 Modularity: 0.41724196 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.6004s
